In [ ]:
import os
import numpy as np
import pickle
import pandas as pd
import json
import re
import glob
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Audio
from IPython.display import display
from scipy.io import wavfile
from itertools import chain 
import string
import Levenshtein as Lev
from itertools import groupby
import scipy.stats as st
from scipy import signal
import nltk
from scipy.stats import wasserstein_distance as wd
from sklearn.metrics.pairwise import euclidean_distances as ed
from pyemd import emd
from hyphenate import hyphenate_word
from itertools import islice 
from collections import OrderedDict
import csv

In [ ]:
#####################################################################################
########### set the Data files paths on Conda Notebook on Ubuntu #################### 

### 
os.chdir('/home/mmm2050/QU_DFKI_Thesis/Experimentation/ASR_Accent_Analysis_De')
main_path=os.getcwd()

if not os.path.exists(main_path+'/Data_results'):
  os.makedirs(main_path+'/Data_results')

if not os.path.exists(main_path+'/Figures_results'):
  os.makedirs(main_path+'/Figures_results')

Data_path=main_path+'/Data/'
Data_results_path=main_path+'/Data_results/'
json_file_path=Data_path+'results.json'
json_file_path_small=Data_path+'results_small.json'
validated_tsv_path=Data_path+'validated.tsv'
validated_tsv_path_small=Data_path+'validated_small.tsv'
 

validated_en_tsv_path=main_path+'/DeepSpeech/data/validated.tsv'
validated_en_tsv_path_small=main_path+'/DeepSpeech/data/validated_small.tsv'

 
os.getcwd()

In [ ]:
#@title read in all three CSV files at once and store them in separate dataframes


# # my_data/all_overlap.txt ==>> test_at.txt
# # transcript ==>> text
# # file==>> audio_filepath
# # all_overlap ==>> dataset_trans_test_all_duration_test_all_output.csv
# ###############################################

# df_at, df_gb, df_it, df_de_al, df_fr, df_de_ni, df_ch, df_de, df_us, df_ca, df_ru = pd.read_csv(['test_at.txt', 'test_gb.txt', 'test_it.txt', 'test_de_al.txt', 'test_fr.txt',
#                                'test_de_ni.txt', 'test_ch.txt', 'test_de.txt', 'test_us.txt', 'test_ca.txt', 'test_ru.txt'] )

MCV_all=pd.DataFrame({})


df_at= pd.read_csv(Data_path+'test_at.txt',delimiter = ", ", header = None ,encoding="utf-8", names=['audio_filepath','transcript','duration'],engine='python' )
df_gb= pd.read_csv(Data_path+'test_gb.txt',delimiter = ", ", header = None ,encoding="utf-8", names=['audio_filepath','transcript','duration'],engine='python' )
df_it= pd.read_csv( Data_path+'test_it.txt',delimiter = ", ", header = None ,encoding="utf-8", names=['audio_filepath','transcript','duration'],engine='python' )
df_de_al= pd.read_csv(Data_path+'test_de_al.txt',delimiter = ", ", header = None ,encoding="utf-8", names=['audio_filepath','transcript','duration'],engine='python' )
df_fr= pd.read_csv(Data_path+'test_fr.txt',delimiter = ", ", header = None ,encoding="utf-8", names=['audio_filepath','transcript','duration'],engine='python' )
df_de_ni= pd.read_csv( Data_path+'test_de_ni.txt',delimiter = ", ", header = None ,encoding="utf-8", names=['audio_filepath','transcript','duration'],engine='python' )
df_ch= pd.read_csv(Data_path+'test_ch.txt',delimiter = ", ", header = None ,encoding="utf-8", names=['audio_filepath','transcript','duration'],engine='python' )
df_de= pd.read_csv(Data_path+'test_de.txt',delimiter = ", ", header = None ,encoding="utf-8", names=['audio_filepath','transcript','duration'],engine='python' )
df_us= pd.read_csv( Data_path+'test_us.txt',delimiter = ", ", header = None ,encoding="utf-8", names=['audio_filepath','transcript','duration'],engine='python' )
df_ca= pd.read_csv(Data_path+'test_ca.txt',delimiter = ", ", header = None ,encoding="utf-8", names=['audio_filepath','transcript','duration'],engine='python' )
df_ru= pd.read_csv(Data_path+'test_ru.txt',delimiter = ", ", header = None ,encoding="utf-8", names=['audio_filepath','transcript','duration'],engine='python' )
 
# combine the dataframes into a single one 
combined_df = pd.concat([df_at, df_gb, df_it, df_de_al, df_fr, df_de_ni, df_ch, df_de, df_us, df_ca, df_ru], ignore_index=True)

dataset_trans_test_all_duration=combined_df

dataset_trans_test_all_duration['audio_filepath'] = dataset_trans_test_all_duration['audio_filepath'].map(lambda x: x.split('.',6)[2])
dataset_trans_test_all_duration['audio_filepath'] = dataset_trans_test_all_duration['audio_filepath'].map(lambda x: x.split('/',6)[3])
dataset_trans_test_all_duration['transcript'] = dataset_trans_test_all_duration['transcript'].map(lambda x: x.split('": "',2)[1])
dataset_trans_test_all_duration['transcript'] = dataset_trans_test_all_duration['transcript'].map(lambda x: x.split('"',2)[0])
dataset_trans_test_all_duration['duration'] = dataset_trans_test_all_duration['duration'].map(lambda x: x.split('": ',2)[1])
dataset_trans_test_all_duration['duration'] = dataset_trans_test_all_duration['duration'].map(lambda x: x.split('}',2)[0])
    

print(type(dataset_trans_test_all_duration),'\n',dataset_trans_test_all_duration.head(10))

print(len(dataset_trans_test_all_duration))
# the complete final dataframe of all Accents txt files
dataset_trans_test_all_duration.to_csv( Data_results_path+'dataset_trans_test_all_duration.csv')

MCV_all['audio_filepath']=dataset_trans_test_all_duration['audio_filepath']
MCV_all['transcript']=dataset_trans_test_all_duration['transcript']

MCV_all.to_csv( Data_results_path+'MCV_all.csv')


dataset_trans_test_all_duration['transcript']

transcripts = list(set(dataset_trans_test_all_duration['transcript'].tolist()))
len(transcripts)
trans_dict = {x:[] for x in transcripts}

# Assign the row['transcript'] value to the 
for index, row in dataset_trans_test_all_duration.iterrows():
    trans_dict[row['transcript']].append(row['audio_filepath'])

In [ ]:
trans_dict['schreib ihr halt ein paar liebe worte rein']